In [2]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [3]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


475.9736399650574

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [5]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


In [3]:
# В качестве policy реализуем верхнюю квантиль
class UCB:
    def __init__(self, c: float):
        self.time = 0 # время учитывается в эксплорейшен части
        self.c = c # будем тюнить баланс эксплорейшен / эксплойтешн
    
    def __call__(self, history: pd.DataFrame):
        self.time += 1
        exploit = history['clicks'] / (history['impressions'] + 1)
        explor = np.sqrt(2 * np.log(self.time) / (history['impressions'] + 1))
        n = np.argmax(exploit + self.c * explor)
        return history.index[n]

In [4]:
for c in [0.001, 0.01, 0.05, 0.1, 0.5, 1.0]:
    print("C = ", c)
    seed = 18475
    np.random.seed(seed=seed)
    start = time.time()
    output = simulation(UCB(c=c), n=200000, seed=seed)
    end = time.time()
    end - start
    print(output['regret'], output['regret']/output['rounds'],  output['total_banners'])


C =  0.001
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
7570.250349143192 0.03785125174571596 184
C =  0.01
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have 

In [5]:
# посерчим в окрестности 0.1 
for c in [0.08, 0.1, 0.15, 0.2, 0.25]:
    print("C = ", c)
    seed = 18475
    np.random.seed(seed=seed)
    start = time.time()
    output = simulation(UCB(c=c), n=200000, seed=seed)
    end = time.time()
    end - start
    print(output['regret'], output['regret']/output['rounds'],  output['total_banners'])

C =  0.08
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
197.72686196531987 0.0009886343098265994 184
C =  0.1
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have

Вывод: наилучший $c = 0.08$, побили бейзлайн почти в 8 раз.